Use this code to help determine parameter values to input into apdb to be used in optimizer code

In [ ]:
#!/usr/bin/env python3
import os
import json
import h5py
import argparse
import pandas as pd
import numpy as np
import tinydb as db
from tinydb.storages import MemoryStorage
import matplotlib.pyplot as plt
# plt.style.use('../clint.mpl')
from matplotlib.colors import LogNorm

# import boost_histogram as bh
# import pickle as pl

from pygama import DataGroup
import pygama.lh5 as lh5
import pygama.analysis.histograms as pgh
import pygama.analysis.peak_fitting as pgf
from pygama.dsp.WaveformBrowser import WaveformBrowser

In [ ]:
bkg_dg = DataGroup('$CAGE_SW/processing/cage.json', load=True)
# bkg_run = 'run==180 and skip==False'
bkg_run = 'run==57 and skip==False'
# bkg_run = 'run==34 and skip==False'
bkg_dg.fileDB.query(bkg_run, inplace=True)
view_cols = ['runtype', 'run', 'cycle', 'startTime', 'runtime', 'threshold']
print(bkg_dg.fileDB[view_cols])

In [ ]:
user = True
# hit = False
hit = True

In [ ]:
# get file list and load background energy data

lh5_dir = bkg_dg.lh5_user_dir if user else bkg_dg.lh5_dir

# if hit files
if hit == True:
    bkg_file_list = lh5_dir + bkg_dg.fileDB['hit_path'] + '/' + bkg_dg.fileDB['hit_file']
    # bkg_df = lh5.load_dfs(bkg_file_list, ['energy', 'trapEmax', 'trapEftp', 'trapEmax_cal', 'trapEftp_cal', 'bl','bl_sig', 'bl_slope', 'lf_max', 'A_10','AoE', 'dcr', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max', 'ts_sec', 'timestamp', 'ts_glo'], 'ORSIS3302DecoderForEnergy/hit')
    bkg_df = lh5.load_dfs(bkg_file_list, ['energy', 'trapEftp', 'trapEftp_cal', 'bl','bl_sig'], 'ORSIS3302DecoderForEnergy/hit')
    
else:
    bkg_file_list = lh5_dir + bkg_dg.fileDB['dsp_path'] + '/' + bkg_dg.fileDB['dsp_file']
    bkg_df = lh5.load_dfs(bkg_file_list, ['energy', 'trapEmax', 'trapEftp', 'bl','bl_sig', 'bl_slope', 'lf_max', 'A_10','AoE', 'dcr', 'tp_0', 'tp_10', 'tp_90', 'tp_50', 'tp_80', 'tp_max', 'timestamp'], 'ORSIS3302DecoderForEnergy/dsp')
    
    print(bkg_file_list)

bkg_rt_min = bkg_dg.fileDB['runtime'].sum()
bkg_u_start = bkg_dg.fileDB.iloc[0]['startTime']
bkg_t_start = pd.to_datetime(bkg_u_start, unit='s') # str
print(f'Background Runtime: {bkg_rt_min} minutes')

In [ ]:
plt.rcParams['figure.figsize'] = [14, 10]

Plot full energy spectrum

In [ ]:
elo, ehi, epb = 0, 5000000, 1000
nbx = int((ehi-elo)/epb)
e_unit = ' raw'

etype = 'energy'

In [ ]:

bkg_ene_hist_raw, bins = np.histogram(bkg_df[etype], bins=nbx, range=([elo, ehi]))

# print(bins)

plt.semilogy(bins[1:], bkg_ene_hist_raw, ds='steps', c='b', lw=1,
            label=f'{etype}, background energy')

plt.xlabel(etype, ha='right', x=1)
plt.ylabel('cts', ha='right', y=1)
plt.ylabel(f'counts/{str(epb)}{e_unit}', fontsize=16)
# plt.xlim(2610, 2620)
# plt.ylim(0.001, 30)
plt.legend()
plt.title(f'Background run, raw energy: {bkg_run}')
plt.show()

Plot 1460 peak location

In [ ]:
k40_elo, k40_ehi, k40_epb = 2610000, 2660000, 1000
k40_nbx = int((k40_ehi-k40_elo)/k40_epb)
e_unit = ' raw'

etype = 'energy'

In [ ]:
k40_bkg_ene_hist_raw, k40_bins = np.histogram(bkg_df[etype], bins=k40_nbx, range=([k40_elo, k40_ehi]))

# print(bins)

plt.semilogy(k40_bins[1:], k40_bkg_ene_hist_raw, ds='steps', c='b', lw=1,
            label=f'{etype}, background energy')

plt.xlabel(etype, ha='right', x=1)
plt.ylabel('cts', ha='right', y=1)
plt.ylabel(f'counts/{str(epb)}{e_unit}', fontsize=16)
# plt.xlim(2610, 2620)
# plt.ylim(0.001, 30)
plt.legend()
plt.title(f'Background run, raw energy: {bkg_run}')
plt.show()

plot 2615 peak 

In [ ]:
tl_elo, tl_ehi, tl_epb = 4710000, 4750000, 1000
tl_nbx = int((tl_ehi-elo)/tl_epb)
e_unit = ' raw'

etype = 'energy'

In [ ]:
tl_bkg_ene_hist_raw, tl_bins = np.histogram(bkg_df[etype], bins=tl_nbx, range=([tl_elo, tl_ehi]))

# print(bins)

plt.semilogy(tl_bins[1:], tl_bkg_ene_hist_raw, ds='steps', c='b', lw=1,
            label=f'{etype}, background energy')

plt.xlabel(etype, ha='right', x=1)
plt.ylabel('cts', ha='right', y=1)
plt.ylabel(f'counts/{str(epb)}{e_unit}', fontsize=16)
# plt.xlim(2610, 2620)
# plt.ylim(0.001, 30)
plt.legend()
plt.title(f'Background run, raw energy: {bkg_run}')
plt.show()

Use Waveform Browser to figure out optimal time bins to optimize DCR and energy traps

In [ ]:
# get raw files for wfs

raw_lh5_dir = bkg_dg.lh5_dir
raw_list = raw_lh5_dir + bkg_dg.fileDB['raw_path'] + '/' + bkg_dg.fileDB['raw_file']

print(raw_list)

In [ ]:
# look at 1460 events

energy = bkg_df[etype]

energy_cut = (energy > 2610000) & (energy < 2650000)

DCR

In [ ]:
dsp_config_file = os.path.expandvars('$CAGE_SW/processing/metadata/config_dsp.json')
browser = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, # Need to include a dsp config file!
                          database={"pz_const":'396.9*us'}, # TODO: use metadata instead of manually defining...
                          waveforms=['wf_blsub', 'wf_pzDCR', 'dcr_trap', 'dcr_trap'], # names of waveforms from dsp config file
                          selection   = energy_cut,           # Apply cut
                          wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                          legend=['Waveform', 'PZ Corrected', "DCR: {dcr:0.2f}"],
#                           legend_opts={'loc':"upper left"},
                          lines=['dcr'], # add hlines and vlines
                          x_lim=(38000, 80000) # x axis range
                         )

In [ ]:
browser.draw_next()
# plt.ylim(0, 100)
# plt.plot(4090)

Energy

In [ ]:
dsp_config_file = os.path.expandvars('$CAGE_SW/processing/metadata/config_dsp.json')
browser2 = WaveformBrowser(raw_list, 'ORSIS3302DecoderForEnergy/raw', dsp_config_file, # Need to include a dsp config file!
                          database={"pz_const":'396.9*us'}, # TODO: use metadata instead of manually defining...
                          waveforms=['wf_blsub', 'wf_pz', 'wf_trap'], # names of waveforms from dsp config file
                          selection   = energy_cut,           # Apply cut
                          wf_styles=[{'linestyle':['-']},{'linestyle':[':']},{'ls':['--']}],
                          legend=['Waveform', 'PZ Corrected',  'trapEftp = {trapEftp:0.2f}'],
#                           legend_opts={'loc':"upper left"},
                          lines=['trapEftp'], # add hlines and vlines
                          x_lim=(0, 80000) # x axis range
                         )

In [ ]:
browser2.draw_next()
# plt.ylim(4000, 4500)